# Inference on New Zealand data and observe effects of epsilon

## Load all information and modify them to the correct format

In [1]:
# Import libraries
import numpy as np
import math
import branchpro
import scipy.stats
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import os
import pandas as pd


In [2]:
# Read serial interval
si_file = 'si-epinow'
path = os.path.join('../data_library/serial_interval', '{}.csv'.format(si_file))
serial_interval = pd.read_csv(path, header=None)
serial_interval = serial_interval.fillna(0)
serial_intervals = serial_interval.values.T

In [31]:
# Read New Zealand data
path = os.path.join('../data_library/covid_new_zealand/NZ.csv')

# Start from day 30, as there are almost no cases before it.
data = pd.read_csv(path)[30:81]

In [32]:
time_key = 'Time'
num_timepoints = (max(data[time_key])-min(data[time_key]))+1
data_times = data[time_key]

# Pad with zeros the time points where we have no information on
# the number of incidences
padded_inc_data = data.set_index(time_key).reindex(
    range(
        min(data_times), max(data_times)+1)
        ).fillna(0).reset_index()
locally_infected_cases = padded_inc_data['Incidence Number']
imported_cases = padded_inc_data['Imported Cases']

start_times = np.arange(1, num_timepoints+1, dtype=int)
times = np.arange(num_timepoints+1)

In [33]:
# Plot (bar chart cases each day)
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=locally_infected_cases,
        name='Local Incidences'
    )
)

fig.add_trace(
    go.Bar(
        x=times,
        y=imported_cases,
        name='Imported Cases'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()

## Comparison of inference results for different epsilon values

In [34]:
# Same inference, but using the LocImpBranchProPosterior
tau = 2
R_t_start = tau+1
a = 1
b = 0.8

# Run inferences for different values of epsilon
column_names = ['Time Points', 'Mean', 'Lower bound CI', 'Upper bound CI', 'Central Probability', 'Epsilon']
epsilon_range = np.arange(-1, 1.1, 0.2)
chosen_times = [5, 10, 15, 20, 25, 30, 40]

all_intervals = pd.DataFrame(columns=column_names)
# Transform our incidence data into pandas dataframes
inc_data = pd.DataFrame(
    {
        'Time': start_times,
        'Incidence Number': locally_infected_cases
    }
)

imported_inc_data = pd.DataFrame(
    {
        'Time': start_times,
        'Incidence Number': imported_cases
    }
)

for epsilon in epsilon_range:
    inference = branchpro.LocImpBranchProPosteriorMultSI(
        inc_data=inc_data,
        imported_inc_data=imported_inc_data,
        epsilon=epsilon,
        daily_serial_intervals=serial_intervals,
        alpha=a,
        beta=1/b)

    inference.run_inference(tau=tau)
    intervals = inference.get_intervals(central_prob=0)
    intervals = intervals.append(inference.get_intervals(central_prob=.25))
    intervals = intervals.append(inference.get_intervals(central_prob=.95))
    temp = inference.get_intervals(central_prob=.95)

    intervals['Epsilon'] = [epsilon] * len(intervals.index)

    intervals_chosen = pd.DataFrame(columns=intervals.columns)
    for time_pt in chosen_times:
        intervals_chosen = intervals_chosen.append(
            intervals.loc[intervals['Time Points']== time_pt])

    all_intervals = all_intervals.append(intervals_chosen)

In [35]:
# Create list of names for x-axis and traces
epsilon_range = np.around(epsilon_range,1)
num_epsilons = len(epsilon_range)
x_name = []
for i in range(num_epsilons):
    x_name.append(str(epsilon_range[i]))
trace_name = ['Day{}'.format(x) for x in chosen_times]

# Sort results
all_intervals.sort_values(by=['Time Points', 'Epsilon'], inplace=True)

# Plot (box plot for inference)
fig = go.Figure()

for i in range(len(trace_name)):
    fig.add_trace(
        go.Box(
            mean=all_intervals.loc[all_intervals['Central Probability']==0]['Mean'][i*num_epsilons:(i+1)*num_epsilons],
            median=all_intervals.loc[all_intervals['Central Probability']==0]['Lower bound CI'][i*num_epsilons:(i+1)*num_epsilons],
            q1=all_intervals.loc[all_intervals['Central Probability']==0.25]['Lower bound CI'][i*num_epsilons:(i+1)*num_epsilons],
            q3=all_intervals.loc[all_intervals['Central Probability']==0.25]['Upper bound CI'][i*num_epsilons:(i+1)*num_epsilons],
            upperfence=all_intervals.loc[all_intervals['Central Probability']==0.95]['Upper bound CI'][i*num_epsilons:(i+1)*num_epsilons],
            lowerfence=all_intervals.loc[all_intervals['Central Probability']==0.95]['Lower bound CI'][i*num_epsilons:(i+1)*num_epsilons],
            x=x_name,
            name=trace_name[i]
        )
    )
fig.update_layout(boxmode='group', title='New Zealand')
fig.show()

In [36]:
# Create list of names for x-axis and traces
group_name = ['Day{}'.format(x) for x in chosen_times]

# Sort results
all_intervals.sort_values(by=['Epsilon','Time Points'], inplace=True)

# Plot (box plot for inference)
fig = go.Figure()

n = len(group_name)
for i in range(len(epsilon_range)):
    fig.add_trace(
        go.Box(
            mean=all_intervals.loc[all_intervals['Central Probability']==0]['Mean'][i*n:(i+1)*n],
            median=all_intervals.loc[all_intervals['Central Probability']==0]['Lower bound CI'][i*n:(i+1)*n],
            q1=all_intervals.loc[all_intervals['Central Probability']==0.25]['Lower bound CI'][i*n:(i+1)*n],
            q3=all_intervals.loc[all_intervals['Central Probability']==0.25]['Upper bound CI'][i*n:(i+1)*n],
            upperfence=all_intervals.loc[all_intervals['Central Probability']==0.95]['Upper bound CI'][i*n:(i+1)*n],
            lowerfence=all_intervals.loc[all_intervals['Central Probability']==0.95]['Lower bound CI'][i*n:(i+1)*n],
            x=group_name,
            name=str(round(epsilon_range[i],1))
        )
    )
fig.update_layout(boxmode='group', title='New Zealand')
fig.show()